# Workspace Access Enforcement

## Overview
This notebook enforces security governance across your Microsoft Fabric tenant. It ensures that specific security groups (e.g., "Fabric Admins") are assigned to every workspace with the correct permissions.

## Modes of Operation
- **Assess Mode** (`MODE = "assess"`): Runs a "dry run" to identify non-compliant workspaces without making changes. It produces a report of what *would* be done.
- **Enforce Mode** (`MODE = "enforce"`): Actively applies permission changes to non-compliant workspaces to bring them into compliance.

## How to Use
1. **Install Package**: Installs the `usf_fabric_monitoring` package.
2. **Configure Credentials**: Loads Service Principal credentials from `.env` or Key Vault.
3. **Define Rules**:
    - `REQUIREMENTS`: Define which groups must have access (e.g., `{"group_name": "Fabric Admins", "access_right": "Admin"}`).
    - `SUPPRESSIONS`: List workspaces to exclude from enforcement.
4. **Run Enforcement**: Execute the notebook. It will scan workspaces and output a summary of compliance status and actions taken.

<span style="color:red">pip install is only required on first run. as a result, it is commented out</span>

In [ ]:
# %pip install /lakehouse/default/Files/usf_fabric_monitoring-0.1.1-py3-none-any.whl --force-reinstall

In [ ]:
from usf_fabric_monitoring.core.workspace_access_enforcer import WorkspaceAccessEnforcer
from usf_fabric_monitoring.core.logger import setup_logging
import logging
import os
from pathlib import Path

In [ ]:
import os
from dotenv import load_dotenv

# --- CREDENTIAL MANAGEMENT ---

# Option 1: Load from .env file in Lakehouse (Easiest migration)
# Upload your .env file to the 'Files' section of your Lakehouse
ENV_PATH = "/lakehouse/default/Files/dot_env_files/.env"
if os.path.exists(ENV_PATH):
    print(f"Loading configuration from {ENV_PATH}")
    # We use the default behavior (override=False) to prefer existing environment variables
    # (e.g., from Fabric Environment settings) over the .env file if they exist.
    load_dotenv(ENV_PATH)
else:
    print(f"Warning: No .env file found at {ENV_PATH}")

# Option 2: Load from Azure Key Vault (Best Practice)
# Uncomment and configure this section to use Azure Key Vault
# try:
#     from notebookutils import mssparkutils
#     KEY_VAULT_NAME = "YourKeyVaultName"
#     os.environ["AZURE_CLIENT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-ID")
#     os.environ["AZURE_CLIENT_SECRET"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Client-Secret")
#     os.environ["AZURE_TENANT_ID"] = mssparkutils.credentials.getSecret(KEY_VAULT_NAME, "Fabric-Tenant-ID")
# except ImportError:
#     pass # Not running in Fabric or notebookutils not available
# except Exception as e:
#     print(f"Key Vault access failed: {e}")

# Verify credentials are present
required_vars = ["AZURE_CLIENT_ID", "AZURE_CLIENT_SECRET", "AZURE_TENANT_ID"]
missing = [v for v in required_vars if not os.getenv(v)]

if missing:
    print(f"❌ Missing required environment variables: {', '.join(missing)}")
else:
    secret = os.getenv("AZURE_CLIENT_SECRET")
    masked_secret = f"{secret[:5]}...{secret[-3:]}" if secret else "None"
    print("✅ Credentials configured successfully")
    print(f"   Client ID: {os.getenv('AZURE_CLIENT_ID')}")
    print(f"   Secret:    {masked_secret} (Verify this matches your .env file)")

In [ ]:
# Configuration
MODE = "assess" # Options: "assess" (Dry Run) or "enforce" (Apply Changes)
FABRIC_ONLY = True # Set to True to only enforce on Fabric/Premium workspaces
OUTPUT_DIR = Path("/lakehouse/default/Files/workspace_access_enforcement")

# Derive DRY_RUN from MODE for safety
DRY_RUN = (MODE.lower() != "enforce")

print(f"🔧 Operation Mode: {MODE.upper()}")
print(f"🛡️  Dry Run: {'ENABLED (No changes will be made)' if DRY_RUN else 'DISABLED (Changes WILL be applied)'}")
print(f"🏭 Fabric Only: {FABRIC_ONLY}")

# --- CONFIGURATION FILES ---
# Upload your 'config' folder to the Lakehouse Files section: /lakehouse/default/Files/config/
CONFIG_DIR = Path("/lakehouse/default/Files/config")
TARGETS_FILE = CONFIG_DIR / "workspace_access_targets.json"
SUPPRESS_FILE = CONFIG_DIR / "workspace_access_suppressions.json"

# Load Access Requirements
if TARGETS_FILE.exists():
    print(f"Loading targets from {TARGETS_FILE}")
    REQUIREMENTS = WorkspaceAccessEnforcer.load_access_requirements(TARGETS_FILE)
else:
    print(f"⚠️ Config file not found at {TARGETS_FILE}. Using defaults.")
    # Fallback: Create AccessRequirement objects manually if file is missing
    from usf_fabric_monitoring.core.workspace_access_enforcer import AccessRequirement
    REQUIREMENTS = (
        AccessRequirement(
            object_id="00000000-0000-0000-0000-000000000000", # Replace with real Group ID
            display_name="Fabric Admins",
            role="Admin"
        ),
    )

# Load Suppressions
if SUPPRESS_FILE.exists():
    print(f"Loading suppressions from {SUPPRESS_FILE}")
    SUPPRESSIONS = WorkspaceAccessEnforcer.load_suppressions(SUPPRESS_FILE)
else:
    print(f"⚠️ Config file not found at {SUPPRESS_FILE}. Using empty defaults.")
    SUPPRESSIONS = WorkspaceAccessEnforcer.load_suppressions(None)

In [ ]:
logger = setup_logging(name="workspace_enforcer", level=logging.INFO)

enforcer = WorkspaceAccessEnforcer(
    access_requirements=REQUIREMENTS,
    suppressions=SUPPRESSIONS,
    dry_run=DRY_RUN,
    logger=logger
)

summary = enforcer.enforce(fabric_only=FABRIC_ONLY)

print(f"Enforcement Summary: {summary}")